In [45]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import math
import skbio
current_file="kmers042720.csv"
from datetime import date
todaydate = date.today()
outputname=("reyni_values" + current_file[:-4] + str(todaydate) + ".csv")

In [46]:
reyni=pd.read_csv(current_file)
div=pd.read_csv(current_file)
with open(current_file) as file:
    header = file.readline().split(",")
uniqueindex = header[0]   

In [47]:
#Preparing Diverity Calculations. Obtaining a clean dataframe without frequencies.
div = div.drop(uniqueindex, axis=1)
div = div.transpose()

In [48]:
print(skbio.diversity.get_alpha_diversity_metrics())
print(skbio.diversity.get_beta_diversity_metrics())

['ace', 'berger_parker_d', 'brillouin_d', 'chao1', 'chao1_ci', 'dominance', 'doubles', 'enspie', 'esty_ci', 'faith_pd', 'fisher_alpha', 'gini_index', 'goods_coverage', 'heip_e', 'kempton_taylor_q', 'lladser_ci', 'lladser_pe', 'margalef', 'mcintosh_d', 'mcintosh_e', 'menhinick', 'michaelis_menten_fit', 'observed_otus', 'osd', 'pielou_e', 'robbins', 'shannon', 'simpson', 'simpson_e', 'singles', 'strong']
['unweighted_unifrac', 'weighted_unifrac']


In [49]:
diversity_results= pd.DataFrame()
diversity_results["Observed_Otus"]=skbio.diversity.alpha_diversity("observed_otus",div, ids=div.index)
diversity_results["Simpsons_Index"]=skbio.diversity.alpha_diversity("simpson",div, ids=div.index)
diversity_results["Simspons_Evenness_Measure_E"]=skbio.diversity.alpha_diversity("simpson_e",div, ids=div.index)
diversity_results["Shannon_Entropy"]=skbio.diversity.alpha_diversity("shannon",div, ids=div.index)
diversity_results["Fisher_Alpha"]=skbio.diversity.alpha_diversity("fisher_alpha",div, ids=div.index)

In [50]:
display(diversity_results)

,Observed_Otus,Simpsons_Index,Simspons_Evenness_Measure_E,Shannon_Entropy,Fisher_Alpha
10A,3013,0.998447,0.213731,10.573545,528.385257
10B,3125,0.998297,0.187958,10.521421,551.517847
11A,2407,0.994521,0.075831,8.980683,395.992617
11B,2194,0.994008,0.076065,8.868474,356.084265
13A,3075,0.998764,0.263150,10.807790,532.012166
13B,3039,0.998894,0.297633,10.874930,532.628552
14A,3023,0.999026,0.339787,10.925950,525.830645
14B,2833,0.998952,0.336884,10.852178,481.715583
15D,2823,0.998833,0.303537,10.785255,488.326515
16A,3066,0.999104,0.364196,10.986159,546.880654


In [51]:
#Convert reyni dataframes to frequencies of each column
reyni.set_index(uniqueindex, inplace=True)
reynisums=pd.Series(reyni.sum())
reynifreq = reyni / reynisums
alphalist = [0,.99999,2,3,4]
reyniresults={}
for column in reynifreq:
    #print(reynifreq[column])
    listresults=[]
    for alpha in alphalist:
        columnsum=0
        for row in reynifreq[column]:
            if row > 0:
                columnsum+=row**alpha
                #print(columnsum)
            else:
                continue
        columnsum=columnsum**(1/(1-alpha))
        listresults.append(columnsum)
    reyniresults[column]=listresults

In [53]:
reynidf = pd.DataFrame.from_dict(data=reyniresults, orient='columns')
reynidfl = pd.DataFrame.from_dict(data=reyniresults, orient='index', columns=alphalist)
reynidft = reynidfl.transpose()
display(reynidfl)
f = open(outputname, "w")
f.write(reynidfl.to_csv(index=False, sep='\t'))
f.close()
print(f"File {outputname} has been succesfully created")

,0.00000,0.99999,2.00000,3.00000,4.00000
10A,3013.0,1523.905391,643.972310,392.194474,311.772596
10B,3125.0,1469.830980,587.367493,360.822013,291.245250
11A,2407.0,505.198758,182.524705,137.753146,123.013927
11B,2194.0,467.395120,166.887429,128.122911,115.727350
13A,3075.0,1792.553081,809.185865,478.438403,375.358101
13B,3039.0,1877.944735,904.507850,535.277424,416.053406
14A,3023.0,1945.543156,1027.174641,611.143064,460.311039
14B,2833.0,1848.559514,954.391001,575.046230,442.060887
15D,2823.0,1764.768930,856.883811,513.409735,401.758055
16A,3066.0,2028.455031,1116.624726,670.613796,507.545842


File reyni_valueskmers0427202020-04-27.csv has been succesfully created
